# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from tensorflow.keras.optimizers import Adam

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [2]:
# set path where C files reside

path = r"storage/linux-kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['relay.c', 'scs.c', '.gitignore', 'regset.c', 'pid_namespace.c', 'kmod.c', 'watchdog.c', 'backtracetest.c', 'gen_kheaders.sh', 'profile.c', 'kthread.c', 'dma.c', 'uid16.c', 'entry', 'uid16.h', 'cpu_pm.c', 'signal.c', 'irq_work.c', 'umh.c', 'smpboot.h', 'rseq.c', 'smp.c', 'watch_queue.c', 'padata.c', 'configs.c', 'audit_tree.c', 'auditsc.c', 'softirq.c', 'user.c', 'kcov.c', 'Makefile', 'module.c', 'static_call.c', 'acct.c', 'extable.c', 'cpu.c', 'workqueue.c', 'module-internal.h', 'futex.c', 'crash_dump.c', 'Kconfig.locks', 'kexec_internal.h', 'usermode_driver.c', 'ptrace.c', 'sched', 'fail_function.c', 'audit.h', 'compat.c', 'trace', 'workqueue_internal.h', 'kprobes.c', 'sys.c', 'ucount.c', 'module_signature.c', 'Kconfig.preempt', 'module_signing.c', 'iomem.c', 'seccomp.c', 'tracepoint.c', 'range.c', 'crash_core.c', 'configs', 'fork.c', 'stackleak.c', 'exit.c', 'kexec_file.c', 'gcov', 'task_work.c', 'resource.c', 'rcu', 'nsproxy.c', 'cgroup', 'torture.c', 'test_kprobes.c', 'kexec_core

In [3]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['relay.c', 'scs.c', 'regset.c', 'pid_namespace.c', 'kmod.c', 'watchdog.c', 'backtracetest.c', 'profile.c', 'kthread.c', 'dma.c', 'uid16.c', 'cpu_pm.c', 'signal.c', 'irq_work.c', 'umh.c', 'rseq.c', 'smp.c', 'watch_queue.c', 'padata.c', 'configs.c', 'audit_tree.c', 'auditsc.c', 'softirq.c', 'user.c', 'kcov.c', 'module.c', 'static_call.c', 'acct.c', 'extable.c', 'cpu.c', 'workqueue.c', 'futex.c', 'crash_dump.c', 'usermode_driver.c', 'ptrace.c', 'fail_function.c', 'compat.c', 'kprobes.c', 'sys.c', 'ucount.c', 'module_signature.c', 'module_signing.c', 'iomem.c', 'seccomp.c', 'tracepoint.c', 'range.c', 'crash_core.c', 'fork.c', 'stackleak.c', 'exit.c', 'kexec_file.c', 'task_work.c', 'resource.c', 'nsproxy.c', 'torture.c', 'test_kprobes.c', 'kexec_core.c', 'auditfilter.c', 'freezer.c', 'stop_machine.c', 'kcmp.c', 'kexec.c', 'up.c', 'audit_watch.c', 'kallsyms.c', 'user-return-notifier.c', 'hung_task.c', 'notifier.c', 'resource_kunit.c', 'cred.c', 'sysctl-test.c', 'ksysfs.c', 'smpboot.c', 'asy

In [4]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [5]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0
#include "audit.h"
#include <linux/fsnotify_backend.h>
#include <linux/namei.h>
#include <linux/mount.h>
#include <linux/kthread.h>
#include <linux/refcount.h>
#include <linux/slab.h>

struct audit_tree;
struct audit_chunk;

struct audit_tree {
	refcount_t count;
	int goner;
	struct audit_chunk *root;
	struct list_head chunks;
	struct list_head rules;
	struct list_head list;
	struct list_head same_root;
	struct rcu_head head;
	char pathname[];
};

struct audit_chunk {
	struct list_head hash;
	unsigned long key;
	struct fsnotify_mark *mark;
	struct list_head trees;		/* with root here */
	int count;
	atomic_long_t refs;
	struct rcu_head head;
	struct node {
		struct list_head list;
		struct audit_tree *owner;
		unsigned index;		/* index; upper bit indicates 'will prune' */
	} owners[];
};

struct audit_tree_mark {
	struct fsnotify_mark mark;
	struct audit_chunk *chunk;
};

static LIST_HEAD(tree_list);
static LIST_HEAD(prune_list);
static struct task_st

In [6]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2218747


In [7]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [8]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 96


## Divide data in input (X) and output (y)

### Create sequences

In [11]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [12]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [13]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [14]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 96)
Shape of y: (133317, 96)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [15]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.5))
model.add(LSTM(128, dropout=0.5))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [16]:
# check model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 128)           115200    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 96)                12384     
                                                                 
Total params: 259,168
Trainable params: 259,168
Non-trainable params: 0
_________________________________________________________________


In [17]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1042/1042 [==============================] - 36s 28ms/step - loss: 3.1782 - acc: 0.1706
Epoch 2/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.8364 - acc: 0.2416
Epoch 3/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.7082 - acc: 0.2690
Epoch 4/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.6467 - acc: 0.2850
Epoch 5/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.6008 - acc: 0.2940
Epoch 6/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.5702 - acc: 0.3017
Epoch 7/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.5514 - acc: 0.3074
Epoch 8/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.5318 - acc: 0.3123
Epoch 9/20
1042/1042 [==============================] - 29s 28ms/step - loss: 2.5108 - acc: 0.3156
Epoch 10/20
1042/1042 [==============================] - 30s 28ms/step - loss: 2.4930 - acc: 0.3218
Epoch 11/

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [18]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0, 10,  0],
       [ 0, 10,  0]])

In [20]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "SIL_TIMER:
		to->si_tid     = from->si_tid;
		to->"
SIL_TIMER:
		to->si_tid     = from->si_tid;
		to->chank->paneion_fork(&task);

	if (!siginfo_struct kthread_syomes *pide, t);
		return ret;
}

/**
 * wask_struct kthread_work *pushing_t *t, struct *tsh_siginfo *work);

	struct kthread_work_pigst *pid;
	struct task_struct *t, int ceser *worker *tork * tree, sizeof(struct padata_st_uid *wouker);

	if (!struct cask_struct *t, csue;
		if (!into)
					    struct *adkt_tate_t *pase_struct *user)
{
	int ret;
}

Xtatic int process = 0;
		fo(isser(struct kthread_work)
{
	struct kthread_wark *pash;
		t musk_struct hash_struct *t = sub_sd_tren);

static void dash_siginfo_tree(cort * ims pending)
{
	struct kthread_worker *pine;
		if (!struct task_struct *tape
{
	struct prdata_work *wsuser_tree * ifst;

	if (!idatat) {
		return -EINVAL;

	struct kthread_croup_task *krk;
}

/**
 * Wssk_compad(&puse > size

In [21]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "lse))
		goto out;

	ret = 0;
	if (unlikely(!list_e"
lse))
		goto out;

	ret = 0;
	if (unlikely(!list_eates);
			work sy = &urdet_cse;
		return ret;
}

static void ced_compar_siginfo *pid;

	if (!sig_locked;

		forch_signel(&tash_signel long *info, sizeof(int csed);
	struct kthread_work_sug_n *pinsin_tark *watch_quece *
				                io ret_cote) {
			eeurn -EINVAL;

	return -EFNVLL;

	if (!signal_cser);

	return 0;
}

/**
 * cssict_chupk(struct pamata_iser *wark = NULL;

	struct pash_struct *tar;

	if (!ns_task(struct padata_struct *pat;

	if (!ss_taik);
		return -EINVAL;
		return s;
}

/**
 * Waichel_str(&puset_mask_tork);
		return -EINVAL;

	struct kthread_prom_tomt *w q;

		tr = tree_tark_dote_dite_rutex_tomk(&padet->sighand->sighond->signal->sigsit_>lock);
		return -EPNVAL;
		if (!struct rchit_trie *posk);
		if (!sig, &cumrent_mask);



static int read *pd;

	if (!read->signo;
			re